For simplicity, and because we wanted to collect data from a large sample regardless of the audit's completion, we drew one round with a high stopping probability.
However, on average fewer ballots need to be sampled when samples are drawn in multiple smaller rounds.
In this notebook we explore how audits would have transpired for the sample we drew, if it had been drawn in smaller rounds.

In [488]:
# Imports
from r2b2.contest import ContestType, Contest
from r2b2.minerva2 import Minerva2
from r2b2.minerva import Minerva
from r2b2.eor_bravo import EOR_BRAVO
from r2b2.so_bravo import SO_BRAVO
import numpy as np

In [489]:
# First, let's set all the same audit parameters and contest information
contest_name = "\nSchool Construction and Renovation Projects"
tally = {'Approve' : 2391, 'Reject' : 1414}
risk_limit = .1
reported_winner = max(tally, key=tally.get) 
winner_votes = tally[reported_winner]
total_relevant = sum(tally.values())
loser_votes = total_relevant - winner_votes
margin = (winner_votes / total_relevant) - (loser_votes / total_relevant)
contest_reported = Contest(total_relevant, 
                            tally, 
                            num_winners=1, 
                            reported_winners=[reported_winner],
                            contest_type=ContestType.PLURALITY)

In [490]:
# Read in the sample from csv file
import pandas as pd
df = pd.read_csv('test_sample.csv')
df.head()

,Unnamed: 0,Yes,No,Irrelevant
0,1,1,0,0
1,2,1,0,0
2,3,1,0,0
3,4,0,1,0
4,5,0,1,0


In [491]:
# Construct the sample_dict expected by r2b2
df_array = df.to_numpy()
sample = {
    'Approve': sum(df_array[:,1]),
    'Reject': sum(df_array[:,2]),
    'Approve_so': df_array[:,1],
    'Reject_so': df_array[:,2],
}
# Useful constant
MAXIMUM_POSSIBLE_SAMPLE = len(sample['Approve_so'])
print(sample)

{'Approve': 66, 'Reject': 50, 'Approve_so': array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1,
       1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1,
       0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0,
       0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0,
       0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0,
       1, 1, 1, 0, 0]), 'Reject_so': array([0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0,
       0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0,
       1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1,
       1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1,
       1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1,
       1, 0, 0, 1, 1])}


In [492]:
# Here's a function that will let us divide the sample that we drew into hypothetical rounds.
def hypothetical_rounds(sample, num_rounds):
    ballots_per_round = len(sample) / num_rounds
    round_schedule = [ballots_per_round]

In [493]:
# Find the new round schedule if we divide the sample into num_rounds hypothetical rounds
num_rounds = 4
ballots_per_round = len(sample['Approve_so']) // num_rounds
marginal_round_schedule = [ballots_per_round] * (num_rounds - 1)
marginal_round_schedule.append(len(sample['Approve_so']) - sum(marginal_round_schedule))
cumulative_round_schedule = [sum(marginal_round_schedule[0:i+1]) for i in range(len(marginal_round_schedule))]
print('Marginal round schedule: {}'.format(marginal_round_schedule))
print('Cumulative round schedule: {}'.format(cumulative_round_schedule))

Marginal round schedule: [28, 28, 28, 31]
Cumulative round schedule: [28, 56, 84, 115]


In [494]:
# Divide the sample according to this round schedule
samples = []
for round_size in cumulative_round_schedule:
    approve_so = sample['Approve_so'][0:round_size]
    approve = sum(approve_so)
    reject_so = sample['Reject_so'][0:round_size]
    reject = sum(reject_so)
    samples.append({'Approve_so':approve_so, 'Approve':approve, 'Reject_so':reject_so, 'Reject':reject})
# Print
for s in samples:
    print(s['Approve'])
    print(s['Reject'])
    print(s['Approve_so'])
    print(s['Reject_so'])

17
11
[1 1 1 0 0 1 1 1 1 0 0 1 1 1 0 1 0 0 1 0 1 1 1 0 0 1 0 1]
[0 0 0 1 1 0 0 0 0 1 1 0 0 0 1 0 1 1 0 1 0 0 0 1 1 0 1 0]
34
22
[1 1 1 0 0 1 1 1 1 0 0 1 1 1 0 1 0 0 1 0 1 1 1 0 0 1 0 1 1 1 0 1 1 0 1 1 1
 0 0 1 0 1 1 1 0 0 1 0 1 1 1 0 0 1 0 1]
[0 0 0 1 1 0 0 0 0 1 1 0 0 0 1 0 1 1 0 1 0 0 0 1 1 0 1 0 0 0 1 0 0 1 0 0 0
 1 1 0 1 0 0 0 1 1 0 1 0 0 0 1 1 0 1 0]
51
33
[1 1 1 0 0 1 1 1 1 0 0 1 1 1 0 1 0 0 1 0 1 1 1 0 0 1 0 1 1 1 0 1 1 0 1 1 1
 0 0 1 0 1 1 1 0 0 1 0 1 1 1 0 0 1 0 1 0 1 0 0 1 0 1 1 1 0 0 1 0 1 1 1 0 0
 1 0 1 1 1 0 1 1 1 1]
[0 0 0 1 1 0 0 0 0 1 1 0 0 0 1 0 1 1 0 1 0 0 0 1 1 0 1 0 0 0 1 0 0 1 0 0 0
 1 1 0 1 0 0 0 1 1 0 1 0 0 0 1 1 0 1 0 1 0 1 1 0 1 0 0 0 1 1 0 1 0 0 0 1 1
 0 1 0 0 0 1 0 0 0 0]
66
50
[1 1 1 0 0 1 1 1 1 0 0 1 1 1 0 1 0 0 1 0 1 1 1 0 0 1 0 1 1 1 0 1 1 0 1 1 1
 0 0 1 0 1 1 1 0 0 1 0 1 1 1 0 0 1 0 1 0 1 0 0 1 0 1 1 1 0 0 1 0 1 1 1 0 0
 1 0 1 1 1 0 1 1 1 1 0 1 0 0 0 0 1 0 1 0 0 1 0 1 1 1 0 0 1 0 1 1 1 1 0 0 1
 1 1 0 0]
[0 0 0 1 1 0 0 0 0 1 1 0 0 0 1 0 1 1 0 1 0 0 0 1 1 

In [495]:
# Now create new audit objects to run this hypothetical audit with each of the BRAVOs and Minervas
minerva2 = ('Minerva 2.0', Minerva2(risk_limit, 1.0, contest_reported))
minerva = ('Minerva', Minerva(risk_limit, 1.0, contest_reported))
so_bravo = ('Selection-Ordered BRAVO', SO_BRAVO(risk_limit, 1.0, contest_reported))
eor_bravo = ('End-of-Round BRAVO', EOR_BRAVO(risk_limit, 1.0, contest_reported))
audits = [minerva2, minerva, so_bravo, eor_bravo]

In [496]:
# Run the hypothetical audits, printing the results
for audit_name, audit in audits:
    print('{}:'.format(audit_name))
    for i, (round_size, round_sample) in enumerate(zip(cumulative_round_schedule, samples)):
        audit.execute_round(round_size, round_sample)
        print('Round {} with round size {} and winner ballots {}: risk {} -- stopped: {}'.format(i+1, round_size, round_sample['Approve'], round(audit.pvalue_schedule[-1], 4), audit.stopped))
        if audit.stopped:
            break

Minerva 2.0:
Round 1 with round size 28 and winner ballots 17: risk 0.2573 -- stopped: False
Round 2 with round size 56 and winner ballots 34: risk 0.1383 -- stopped: False
Round 3 with round size 84 and winner ballots 51: risk 0.0743 -- stopped: True
Minerva:
Round 1 with round size 28 and winner ballots 17: risk 0.2573 -- stopped: False
Round 2 with round size 56 and winner ballots 34: risk 0.1284 -- stopped: False
Round 3 with round size 84 and winner ballots 51: risk 0.0867 -- stopped: True
Selection-Ordered BRAVO:
Round 1 with round size 28 and winner ballots 17: risk 0.5374 -- stopped: False
Round 2 with round size 56 and winner ballots 34: risk 0.1552 -- stopped: False
Round 3 with round size 84 and winner ballots 51: risk 0.0963 -- stopped: True
End-of-Round BRAVO:
Round 1 with round size 28 and winner ballots 17: risk 0.5374 -- stopped: False
Round 2 with round size 56 and winner ballots 34: risk 0.2888 -- stopped: False
Round 3 with round size 84 and winner ballots 51: risk 0

In [497]:
# Here is a function to run any hypothetical round schedule and print the results
def hypothetical_round_schedule(sample, round_schedule):
    # Divide the sample according to this round schedule
    samples = []
    for round_size in round_schedule:
        approve_so = sample['Approve_so'][0:round_size]
        approve = sum(approve_so)
        reject_so = sample['Reject_so'][0:round_size]
        reject = sum(reject_so)
        samples.append({'Approve_so':approve_so, 'Approve':approve, 'Reject_so':reject_so, 'Reject':reject})
    # Now create new audit objects to run this hypothetical audit with each of the BRAVOs and Minervas
    minerva2 = ('Minerva 2.0', Minerva2(risk_limit, 1.0, contest_reported))
    minerva = ('Minerva', Minerva(risk_limit, 1.0, contest_reported))
    so_bravo = ('Selection-Ordered BRAVO', SO_BRAVO(risk_limit, 1.0, contest_reported))
    eor_bravo = ('End-of-Round BRAVO', EOR_BRAVO(risk_limit, 1.0, contest_reported))
    audits = [minerva2, minerva, so_bravo, eor_bravo]
    # Run the hypothetical audits, printing the results
    for audit_name, audit in audits:
        print('{}:'.format(audit_name))
        for i, (round_size, round_sample) in enumerate(zip(round_schedule, samples)):
            audit.execute_round(round_size, round_sample)
            print('Round {}: {} total, {} winner: risk {} -- stopped: {}'.format(i+1, round_size, round_sample['Approve'], round(audit.pvalue_schedule[-1], 4), audit.stopped))
            if audit.stopped:
                break

In [498]:
# Let's try the function above
hypothetical_round_schedule(sample, [20, 40, 100])

Minerva 2.0:
Round 1: 20 total, 12 winner: risk 0.3626 -- stopped: False
Round 2: 40 total, 25 winner: risk 0.1749 -- stopped: False
Round 3: 100 total, 58 winner: risk 0.0801 -- stopped: True
Minerva:
Round 1: 20 total, 12 winner: risk 0.3626 -- stopped: False
Round 2: 40 total, 25 winner: risk 0.143 -- stopped: False
Round 3: 100 total, 58 winner: risk 0.1181 -- stopped: False
Selection-Ordered BRAVO:
Round 1: 20 total, 12 winner: risk 0.6917 -- stopped: False
Round 2: 40 total, 25 winner: risk 0.1957 -- stopped: False
Round 3: 100 total, 58 winner: risk 0.0943 -- stopped: True
End-of-Round BRAVO:
Round 1: 20 total, 12 winner: risk 0.6917 -- stopped: False
Round 2: 40 total, 25 winner: risk 0.283 -- stopped: False
Round 3: 100 total, 58 winner: risk 0.4529 -- stopped: False


In [499]:
# While Minerva 1.0 requires predetermined round sizes, Minerva 2.0 could run picking round sizes that achieve some desired stopping probability
# Here's a similar hypothetical round schedule function but now round sizes are determined by some stopping probability
def hypothetical_by_sprob(sample, sprob):
    # Now create new audit objects to run this hypothetical audit with each of the BRAVOs and Minervas
    minerva2 = ('Minerva 2.0', Minerva2(risk_limit, 1.0, contest_reported))
    minerva = ('Minerva', Minerva(risk_limit, 1.0, contest_reported))
    so_bravo = ('Selection-Ordered BRAVO', SO_BRAVO(risk_limit, 1.0, contest_reported))
    eor_bravo = ('End-of-Round BRAVO', EOR_BRAVO(risk_limit, 1.0, contest_reported))
    audits = [minerva2, minerva, so_bravo, eor_bravo]
    # Run the hypothetical audits, printing the results
    for audit_name, audit in audits:
        print('{}:'.format(audit_name))
        round_num = 0
        while True:
            round_num += 1
            # Determine the next round size
            if audit_name == 'Minerva' and round_num > 1:
                round_size = int(round_size + round_size * 1.5)
            else:
                round_size = audit.next_sample_size(sprob)
            if round_size > MAXIMUM_POSSIBLE_SAMPLE:
                print('Next round size would be {} which exceeds the sample we drew of {}'.format(round_size, MAXIMUM_POSSIBLE_SAMPLE))
                break

            # Get the sample for this round size
            approve_so = sample['Approve_so'][0:round_size]
            approve = sum(approve_so)
            reject_so = sample['Reject_so'][0:round_size]
            reject = sum(reject_so)
            cur_sample = {'Approve_so':approve_so, 'Approve':approve, 'Reject_so':reject_so, 'Reject':reject}

            # Execute the round
            audit.execute_round(round_size, cur_sample)
            print('Round {}: {} total, {} winner: risk {} -- stopped: {}'.format(i+1, round_size, cur_sample['Approve'], round(audit.pvalue_schedule[-1], 4), audit.stopped))
            if audit.stopped:
                break

In [500]:
# Try out this new function
hypothetical_by_sprob(sample, .5)

Minerva 2.0:
Round 4: 44 total, 28 winner: risk 0.0918 -- stopped: True
Minerva:
Round 4: 44 total, 28 winner: risk 0.0918 -- stopped: True
Selection-Ordered BRAVO:
Round 4: 52 total, 32 winner: risk 0.252 -- stopped: False
Round 4: 68 total, 40 winner: risk 0.0921 -- stopped: True
End-of-Round BRAVO:
Round 4: 70 total, 41 winner: risk 0.4655 -- stopped: False
Round 4: 109 total, 63 winner: risk 0.4734 -- stopped: False
Next round size would be 148 which exceeds the sample we drew of 115
